In [ ]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, cv2

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.projects.deeplab import add_deeplab_config
from detectron2.data import MetadataCatalog, DatasetCatalog

In [5]:
import fiftyone.zoo as foz
dataset = foz.load_zoo_dataset(
    "open-images-v6",
    split="validation",
    classes=["Vehicle registration plate"],
    label_types=["segmentations"],
    label_field="segmentations",
)

 100% |███████████████████| 287/287 [1.4m elapsed, 0s remaining, 5.4 files/s]      
Dataset info written to 'C:\Users\miner\fiftyone\open-images-v6\info.json'
Loading 'open-images-v6' split 'validation'
 100% |█████████████████| 287/287 [5.7s elapsed, 0s remaining, 56.0 samples/s]      
Dataset 'open-images-v6-validation' created


In [6]:
dataset

Name:        open-images-v6-validation
Media type:  image
Num samples: 287
Persistent:  False
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    segmentations:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)

In [16]:
import fiftyone as fo
from fiftyone.utils.data.importers import ImageSegmentationDirectoryImporter
from pathlib import Path
imfolder = 'C:\\Users\\miner\\OneDrive - University of North Carolina at Chapel Hill\\Bear Lab\\optotaxis calibration\\data\\segmentation_iteration_testing\\iter4\\round1\\images'
maskfolder = Path('C:/Users/miner/OneDrive - University of North Carolina at Chapel Hill/Bear Lab/optotaxis calibration/data/segmentation_iteration_testing/iter4/round1/masks')
imp = ImageSegmentationDirectoryImporter(data_path=imfolder,labels_path=maskfolder)
dataset = fo.Dataset.from_importer(imp)

#directory importer puts mask as ground truth, rename to "segmentations" to match demo
dataset.rename_sample_field("ground_truth","segmentations")

import fiftyone.utils.random as four

four.random_split(dataset, {"train": 0.8, "val": 0.2})

 100% |███████████████████| 56/56 [36.0ms elapsed, 0s remaining, 1.6K samples/s]      


In [18]:
dataset

Name:        2025.04.19.18.22.07
Media type:  image
Num samples: 56
Persistent:  False
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    segmentations:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Segmentation)

In [20]:
samp = next(iter(dataset))

In [23]:
samp.segmentations

<Segmentation: {
    'id': '6804220f4fe1c5ab5c34b6b7',
    'tags': [],
    'mask': None,
    'mask_path': 'C:\\Users\\miner\\OneDrive - University of North Carolina at Chapel Hill\\Bear Lab\\optotaxis calibration\\data\\segmentation_iteration_testing\\iter4\\round1\\masks\\migration1_s10_t6.TIF',
}>

In [24]:
samp["id"]

'6804220f4fe1c5ab5c34b6b8'

In [25]:
def semantic_segmentation_to_detectron2(samples:fo.Dataset,seg_field="segmentations"):
    samples.compute_metadata()

    dataset_dicts = []
    for sample in samples.select_fields(["id", "filepath", "metadata", seg_field]):
        height = sample.metadata["height"]
        width = sample.metadata["width"]
        record = {}
        record["file_name"] = sample.filepath
        record["image_id"] = sample.id
        record["height"] = height
        record["width"] = width
        record["sem_seg_file_name"] = samp[seg_field].mask_path
 
            # objs = []
            # for det in sample[seg_field].detections:
            #     tlx, tly, w, h = det.bounding_box
            #     bbox = [int(tlx*width), int(tly*height), int(w*width), int(h*height)]
            #     fo_poly = det.to_polyline()
            #     poly = [(x*width, y*height) for x, y in fo_poly.points[0]]
            #     poly = [p for x in poly for p in x]
            #     obj = {
            #         "bbox": bbox,
            #         "bbox_mode": BoxMode.XYWH_ABS,
            #         "segmentation": [poly],
            #         "category_id": 0,
            #     }
            #     objs.append(obj)

            # record["annotations"] = objs
        dataset_dicts.append(record)

    return dataset_dicts


for d in ["train", "val"]:
    view = dataset.match_tags(d)
    DatasetCatalog.register("fiftyone_" + d, lambda view=view: semantic_segmentation_to_detectron2(view))
    MetadataCatalog.get("fiftyone_" + d).set(stuff_classes=["cytoplasm"])


In [35]:
Path(".").absolute()

WindowsPath('c:/Users/miner/Documents/Github/cell_segmentation_project')

In [40]:
from Mask2Former.mask2former import add_maskformer2_config
cityscapes_config = r"C:\Users\miner\Documents\Github\cell_segmentation_project\Mask2Former\configs\cityscapes\semantic-segmentation\swin\maskformer2_swin_tiny_bs16_90k.yaml"
cfg = get_cfg()
add_deeplab_config(cfg)
add_maskformer2_config(cfg)
cityscapes_config_folder = Path(cityscapes_config).parent
cwd = Path(".").absolute()
try:
    %cd {cityscapes_config_folder}
    cfg.merge_from_file(cityscapes_config)
finally:
    %cd {cwd}

ModuleNotFoundError: 

Please compile MultiScaleDeformableAttention CUDA op with the following commands:
	`cd mask2former/modeling/pixel_decoder/ops`
	`sh make.sh`
